In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
#pyspark init
builder = SparkSession.builder\
            .appName('creditcard_highAPI')\
            .config("spark.driver.memory", "4g")\
            .config("spark.executor.memory", "4g")
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/11 14:45:12 WARN Utils: Your hostname, minhnhat resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/11 14:45:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 14:45:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read input file
raw_df = spark.read.csv('creditcard.csv', header=True, inferSchema=True)

## Preprocessing

In [4]:
#Extract features name
cols = [col for col in raw_df.columns if col != 'Class']

assembler = VectorAssembler(inputCols=cols, outputCol= 'features')
data = assembler.transform(raw_df).select('features', 'Class')

In [5]:
#Split data into training and testing sets

train_size = 0.8
train, test = data.randomSplit([train_size, 1 - train_size], seed=24)

## Fit the model

In [6]:
classifier = LogisticRegression(featuresCol= 'features', labelCol= 'Class')
model = classifier.fit(train)

25/04/11 14:45:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [7]:
# Display model coefficients and intercept
print("Coefficients: ", model.coefficients)
print("Intercept: ", model.intercept)

Coefficients:  [-2.250504196047376e-06,0.0688645266877948,0.07016448431901465,-0.00576464203873841,0.7462871699325018,0.16702718100079336,-0.13202244921429607,-0.05384541515722778,-0.15093111693791111,-0.19636361746967998,-0.8947473003909194,-0.051147766266498085,0.14343834011779408,-0.36193038240685266,-0.4824931278273971,-0.0799749693197667,-0.2655650567516894,-0.04341991301252665,0.08986194477065233,-0.012540904506693915,-0.5037987413317934,0.37225478970799897,0.5428130509594019,-0.06690468366843587,0.2203392157705258,-0.16529865741373223,0.07181658282365695,-0.8698200135552416,-0.293981762874031,0.0009945996705770465]
Intercept:  -8.611743647628357


In [8]:
summary = model.summary
print('Accuracy: ', summary.accuracy)
print('AUC: ', summary.areaUnderROC)
print('Precision: ', summary.precisionByLabel)
print('Recall: ', summary.recallByLabel)

Accuracy:  0.9991968329763702


AUC:  0.9814094322801267
Precision:  [0.9993540591013952, 0.8681318681318682]
Recall:  [0.9998417331973412, 0.6171875]


## Evaluate the model on the test set

In [9]:
eval = model.transform(test)

In [10]:
binary_evaluator = BinaryClassificationEvaluator(labelCol="Class", rawPredictionCol="rawPrediction")
multi_evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction")

accuracy = multi_evaluator.setMetricName("accuracy").evaluate(eval),
auc = binary_evaluator.evaluate(eval)


precision_per_class = []
recall_per_class = []

for label in [0.0, 1.0]: 
    precision = multi_evaluator.setMetricName("precisionByLabel").setMetricLabel(label).evaluate(eval)
    recall = multi_evaluator.setMetricName("recallByLabel").setMetricLabel(label).evaluate(eval)
    precision_per_class.append(precision)
    recall_per_class.append(recall)

print(f"Accuracy: {accuracy}")
print(f"AUC: {auc}")
print(f"Precision per class: {precision_per_class}")
print(f"Recall per class: {recall_per_class}")


Accuracy: (0.9991572885759933,)
AUC: 0.9635123197285692
Precision per class: [0.9992265912007171, 0.9411764705882353]
Recall per class: [0.9999296406395666, 0.5925925925925926]


In [11]:
#Stop the Spark session
spark.stop()